In [1]:
import os
from pathlib import Path

In [13]:
%pwd

'c:\\Users\\Hp\\Videos\\classification implementation - machine learning with MLFlow'

In [3]:
os.chdir('../')

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Arshavin023/Machine-Learning-Web-Application-for-Credit-Risk-Assessment.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Arshavin023"
os.environ["MLFLOW_TRACKING_PASSWORD"]="3a6add494bc8a69b0623dd2f8030d353dde15622"


In [ ]:
# setx MLFLOW_TRACKING_URI "https://dagshub.com/Arshavin023/Machine-Learning-Web-Application-for-Credit-Risk-Assessment.mlflow"
# setx MLFLOW_TRACKING_USERNAME "Arshavin023"
# setx MLFLOW_TRACKING_PASSWORD 

In [50]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [51]:
## create configuration manager
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [52]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/Arshavin023/Machine-Learning-Web-Application-for-Credit-Risk-Assessment.mlflow",
        )

        return model_evaluation_config

In [53]:
import os
import pandas as pd
from sklearn.metrics import (precision_score, recall_score, 
                             f1_score, accuracy_score, roc_auc_score,
                             roc_curve)
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlProject.utils.common import feature_processor

In [54]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, predicted):
        precision = precision_score(actual,predicted)
        recall = recall_score(actual, predicted)
        f1_score = (2*precision*recall)/(precision+recall)

        return precision, recall, f1_score
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        preprocessor = feature_processor()
        test_x = test_data.drop([self.config.target_column],axis=1)
        test_x = preprocessor.fit_transform(test_x)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (precision, recall, f1_score) = self.eval_metrics(test_y,predicted_qualities)

            # Sending metrics as local

            scores = {"precision": precision, "recall":recall, "f1_score":f1_score}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1_score)

            # Model register does not work with file store

            if tracking_uri_type_store != 'file':

                mlflow.sklearn.log_model(model, 'model', registered_model_name = 'XGBClassifier')
            
            else:
                mlflow.sklearn.log_model(model,'model')


In [55]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-09-27 12:57:50,102: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-27 12:57:50,120: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-27 12:57:50,127: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-27 12:57:50,132: INFO: common: created directory at: artifacts]
[2023-09-27 12:57:50,137: INFO: common: created directory at: artifacts/model_evaluation]
[2023-09-27 12:57:50,666: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'XGBClassifier' already exists. Creating a new version of this model...
2023/09/27 12:58:17 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: XGBClassifier, version 3
Created version '3' of model 'XGBClassifier'.
